Se ha guardado en la raiz del proyecto un archivo llamado `raw_data.zip` que sirve como respaldo en caso que el servidor de la API falle.

# Importar librerias y dataframes

In [ ]:
from dataextraction import * # importa todas las variables del script dataextraction.py que contiene los dataframes

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

Hay muchos dataframes para revisar, el siguiente ciclo sirve para seguir la pista de los dataframes importados.

In [3]:
imported_variables = [name for name in dir() if name.endswith('df')] # comprehension list de df's que terminan en 'df'

for variable in imported_variables:
    print(variable)


acceso_internet_fijo_por_rangos_velocidad_bajada_provincia_df
acceso_internet_fijo_por_tecnologia_provincia_df
accesos_banda_ancha_banda_angosta_por_provincia_df
accesos_internet_fijo_por_tecnología_localidad_df
accesos_internet_fijo_por_velocidad_bajada_provincia_df
accesos_internet_fijo_por_velocidad_localidad_df
conectividad_al_servicio_de_internet_df
indicadores_macroeconomicos_df
ingresos_por_operación_servicio_internet_fijo_df
listado_localidades_conectividad_internet_df
penetracion_internet_fijo_por_provincia_cada_100_habitantes_df
penetracion_internet_fijo_por_provincia_cada_100_hogares_df
penetracion_nacional_internet_fijo_df
total_nacional_accesos_internet_fijo_por_banda_ancha_banda_angosta_df
total_nacional_accesos_internet_fijo_por_tecnologia_df
total_nacional_accesos_internet_fijo_por_velocidad_bajada_df
velocidad_media_bajada_internet_nacional_df
velocidad_media_bajada_internet_por_provincia_df


# Preparando el data set

## Uniendo tablas para resumir algunas tablas

Los dataframes `penetracion_internet_fijo_por_provincia_cada_100_habitantes_df` y `penetracion_internet_fijo_por_provincia_cada_100_hogares_df` exactamente iguales excepto por una columna, entonces pueden ser unidos en una tabla. Lo podemos hacer tambien con velocidad promedio.

In [4]:
# chequeando si son los mismo df's excepto por la ultima columna

penetracion_internet_fijo_por_provincia_cada_100_habitantes_df.drop(columns="Accesos por cada 100 hab") \
.equals(penetracion_internet_fijo_por_provincia_cada_100_hogares_df.drop(columns="Accesos por cada 100 hogares"))

True

### Datos regionales

In [51]:
penetracion_velocidad_regional_internet_fijo_df = penetracion_internet_fijo_por_provincia_cada_100_hogares_df \
.merge(penetracion_internet_fijo_por_provincia_cada_100_habitantes_df, how="inner", on=["Año", "Trimestre", "Provincia"]) \
.merge(velocidad_media_bajada_internet_por_provincia_df, how="inner", on=["Año", "Trimestre", "Provincia"])

penetracion_velocidad_regional_internet_fijo_df.head()

,Año,Trimestre,Provincia,Accesos por cada 100 hogares,Accesos por cada 100 hab,Mbps (Media de bajada)
0,2022,3,Buenos Aires,78.11,26,70.19
1,2022,3,Capital Federal,122.28,50,101.05
2,2022,3,Catamarca,65.33,17,60.99
3,2022,3,Chaco,43.86,12,53.21
4,2022,3,Chubut,84.38,26,15.57


In [94]:
# chequeando por duplicados causados por mala escritura
penetracion_velocidad_regional_internet_fijo_df["Provincia"].unique()

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Córdoba', 'Corrientes', 'Entre Ríos', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén',
       'Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucumán'],
      dtype=object)

### Datos nacionales

In [7]:
# la ultima columna es redundante porque ya existe la columna Trimestre
penetracion_nacional_internet_fijo_df.head()

,Año,Trimestre,Accesos por cada 100 hogares,Accesos por cada 100 hab,Periodo
0,2022,3,76.64,23.95,Jul-Sept 2022
1,2022,2,75.97,23.72,Abr-Jun 2022
2,2022,1,73.88,23.05,Ene-Mar 2022
3,2021,4,73.18,22.81,Oct-Dic 2021
4,2021,3,70.58,21.98,Jul-Sept 2021


In [59]:
# la columna Trimestre no es única en la tabla de velocidad promedio, por eso necesita 'slicing' para ignorar la ultima columna

penetracion_velocidad_nacional_internet_fijo_df = penetracion_nacional_internet_fijo_df.drop(columns="Periodo") \
.merge(velocidad_media_bajada_internet_nacional_df.iloc[:,0:3], how="inner", on=["Año", "Trimestre"]) 

penetracion_velocidad_nacional_internet_fijo_df.head()

,Año,Trimestre,Accesos por cada 100 hogares,Accesos por cada 100 hab,Mbps (Media de bajada)
0,2022,3,76.64,23.95,62.46
1,2022,2,75.97,23.72,58.44
2,2022,1,73.88,23.05,55.11
3,2021,4,73.18,22.81,52.34
4,2021,3,70.58,21.98,48.46


In [8]:
penetracion_nacional_internet_fijo_df = penetracion_nacional_internet_fijo_df.drop(columns="Periodo")

### Grabando en archivos CSV

In [60]:
penetracion_velocidad_nacional_internet_fijo_df.to_csv('../processed_data/penetracion_velocidad_nacional_internet_fijo.csv', index=False)

In [61]:
penetracion_velocidad_regional_internet_fijo_df.to_csv('../processed_data/penetracion_velocidad_regional_internet_fijo.csv', index=False)

### Repitiendo par datos economicos

Ahora se crea un tabla para ingresos y datos economicos. En este dataframe no hay datos economicos despues 2018, por eso aparecen valores nulos en su lugar.

In [19]:
datos_economicos = indicadores_macroeconomicos_df \
.merge(ingresos_por_operación_servicio_internet_fijo_df.drop(columns="Periodo"), how="right", on=["Año", "Trimestre"])

datos_economicos

,Año,Trimestre,IPC US,IPC AR,USDARS oficial,USDARS blue,PBI millones,Ingresos (miles de pesos)
0,2022,3,NaN,NaN,NaN,NaN,NaN,"67,055,930"
1,2022,2,NaN,NaN,NaN,NaN,NaN,"60,335,724"
2,2022,1,NaN,NaN,NaN,NaN,NaN,"55,589,997"
3,2021,4,NaN,NaN,NaN,NaN,NaN,"45,467,887"
4,2021,3,NaN,NaN,NaN,NaN,NaN,"42,999,944"
5,2021,2,NaN,NaN,NaN,NaN,NaN,"38,239,667"
6,2021,1,NaN,NaN,NaN,NaN,NaN,"36,676,371"
7,2020,4,NaN,NaN,NaN,NaN,NaN,"33,539,703"
8,2020,3,NaN,NaN,NaN,NaN,NaN,"31,997,445"
9,2020,2,NaN,NaN,NaN,NaN,NaN,"32,102,476"


In [22]:
datos_economicos = datos_economicos.replace(",", "", regex=True) # need regex=True to find commas into the numbers

In [23]:
datos_economicos.to_csv('../processed_data/datos_economicos.csv', index=False)

## Rango de velocidad de bajada

### Datos Regionales

In [ ]:
acceso_internet_fijo_por_rangos_velocidad_bajada_provincia_df.head()

,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,2022,3,Buenos Aires,"29,985","27,709","290,315","297,915","267,044","124,190","3,618,689",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,3,Capital Federal,517,"5,742","34,371","67,829","51,946","28,692","1,253,105",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,3,Catamarca,71,384,"3,107","5,389","5,099","3,737","50,298",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022,3,Chaco,461,987,"16,782","18,938","8,049","15,828","79,390",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022,3,Chubut,109,"1,444","45,707","30,940","34,682","15,309","17,563",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cada intervalo de velocidad esta establecido como atributo, es posible que se saque mayor provecho 'pivoteando' esas columnas hacia las filas.

In [ ]:
acceso_internet_fijo_por_rangos_velocidad_bajada_provincia_df_melted = (
      acceso_internet_fijo_por_rangos_velocidad_bajada_provincia_df.drop(columns="Total")
      .melt(
            id_vars=['Año', 'Trimestre', "Provincia"], # variables que quedaran fijas
            var_name='Rango Velocidad', # nombre de nueva columna que recibe los nombres de las columnas a pivotear
            value_name='Accesos' # nombre de la columna que guardara los valores provenientes de las columnas a pivotear.
            )
              ) 

In [ ]:
acceso_internet_fijo_por_rangos_velocidad_bajada_provincia_df_melted.dropna(inplace=True)

In [ ]:
acceso_internet_fijo_por_rangos_velocidad_bajada_provincia_df_melted = \
acceso_internet_fijo_por_rangos_velocidad_bajada_provincia_df_melted.replace(",", "", regex=True) 

In [ ]:
# hay duplicados por errores de escritura, se borran los espacios al principio y al final usando strip()

acceso_internet_fijo_por_rangos_velocidad_bajada_provincia_df_melted["Provincia"].unique()

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Córdoba', 'Corrientes', 'Entre Ríos', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén',
       'Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucumán',
       ' Buenos Aires ', ' Capital Federal ', ' Catamarca ', ' Chaco ',
       ' Chubut ', ' Córdoba ', ' Corrientes ', ' Entre Ríos ',
       ' Formosa ', ' Jujuy ', ' La Pampa ', ' La Rioja ', ' Mendoza ',
       ' Misiones ', ' Neuquén ', ' Río Negro ', ' Salta ', ' San Juan ',
       ' San Luis ', ' Santa Cruz ', ' Santa Fe ',
       ' Santiago Del Estero ', ' Tierra Del Fuego ', ' Tucumán '],
      dtype=object)

In [ ]:
acceso_internet_fijo_por_rangos_velocidad_bajada_provincia_df_melted["Provincia"] = [x.strip() for x in acceso_internet_fijo_por_rangos_velocidad_bajada_provincia_df_melted["Provincia"]]

In [ ]:
acceso_internet_fijo_por_rangos_velocidad_bajada_provincia_df_melted.to_csv('../processed_data/rango_velocidad_regional_unpivot.csv', index=False)

### Datos nacionales

Existe un dataframe con datos similares pero a nivel nacional, pero agrupando columnas de la anterior tabla se llega al mismo resultado y prescindir de guardar un nuevo archivo.

In [7]:
rango_velocidad_nacional_unpivot = acceso_internet_fijo_por_rangos_velocidad_bajada_provincia_df_melted.groupby(["Año", "Trimestre", "Rango Velocidad"])['Accesos'].sum().reset_index().sort_values(by="Accesos", ascending=False)  
rango_velocidad_nacional_unpivot

,Año,Trimestre,Rango Velocidad,Accesos
275,2022,3,+ 30 Mbps,6995750
267,2022,2,+ 30 Mbps,6741922
259,2022,1,+ 30 Mbps,6336187
251,2021,4,+ 30 Mbps,6032322
16,2014,3,+ 1 Mbps - 6 Mbps,5153437
...,...,...,...,...
39,2015,1,OTROS,0
87,2016,3,OTROS,0
23,2014,3,OTROS,0
15,2014,2,OTROS,0


Teniendo los rango de velocidad, el siguiente dataframe no es necesario. Ademas, es mas útil analizar rangos de velocidad que hacer por numeros especificos. Por ejemplo, no es muy útil visualizar la diferencia de accesos entre 0.5 y 0.512 Mbps.

In [22]:
accesos_internet_fijo_por_velocidad_bajada_provincia_df.head()

,Año,Trimestre,Provincia,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",...,70 Mbps,71 Mbps,75 Mbps,77 Mbps,78 Mbps,80 Mbps,81 Mbps,83 Mbps,82 Mbps,86 Mbps
0,2022,3,Buenos Aires,"65,821",19,- 0,- 0,"29,774",192,"4,611",...,"3,163",- 0,"98,387",- 0,2,223,- 0,1,- 0,- 0
1,2022,3,Capital Federal,"105,477",31,- 0,- 0,451,35,"1,307",...,3,- 0,"30,482",- 0,- 0,8,- 0,- 0,- 0,- 0
2,2022,3,Catamarca,"2,208",- 0,- 0,- 0,34,37,- 0,...,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0
3,2022,3,Chaco,"3,711",- 0,225,111,234,2,- 0,...,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0
4,2022,3,Chubut,"20,024",8,- 0,- 0,69,32,836,...,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0


## Type of connection

In [ ]:
total_nacional_accesos_internet_fijo_por_banda_ancha_banda_angosta_df

,Año,Trimestre,Banda ancha fija,Dial up,Total,Periodo
0,2022,3,"11,078,691","12,437","11,091,128",Jul-Sept 2022
1,2022,2,"10,946,248","12,436","10,958,684",Abr-Jun 2022
2,2022,1,"10,611,390","12,619","10,624,009",Ene-Mar 2022
3,2021,4,"10,476,933","12,861","10,489,794",Oct-Dic 2021
4,2021,3,"10,075,184","10,357","10,085,541",Jul-Sept 2021
5,2021,2,"9,852,702","10,382","9,863,084",Abr-Jun 2021
6,2021,1,"9,637,956","10,016","9,647,972",Ene-Mar 2021
7,2020,4,"9,561,546","10,016","9,571,562",Oct-Dic 2020
8,2020,3,"9,346,183","10,016","9,356,199",Jul-Sept 2020
9,2020,2,"9,021,040","10,016","9,031,056",Abr-Jun 2020


In [ ]:
accesos_banda_ancha_banda_angosta_por_provincia_df

,Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total
0,2022,3,Buenos Aires,"4,715,469","6,199","4,721,668"
1,2022,3,Capital Federal,"1,545,534","2,145","1,547,679"
2,2022,3,Catamarca,"70,292",1,"70,293"
3,2022,3,Chaco,"144,141",5,"144,146"
4,2022,3,Chubut,"164,874",904,"165,778"
...,...,...,...,...,...,...
835,2014,1,Santa Cruz,"26,304",460,"26,764"
836,2014,1,Santa Fe,"506,000",612,"506,612"
837,2014,1,Santiago Del Estero,"37,124",9,"37,133"
838,2014,1,Tierra Del Fuego,"27,272",766,"28,038"


In [59]:
total_nacional_accesos_internet_fijo_por_tecnologia_df.head()

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo
0,2022,3,"1,395,277","6,031,970","2,871,541","557,110","235,230","11,091,128",Jul-Sept 2022
1,2022,2,"1,468,333","5,979,214","2,723,285","556,243","231,609","10,958,684",Abr-Jun 2022
2,2022,1,"1,533,240","6,073,426","2,219,533","545,814","251,996","10,624,009",Ene-Mar 2022
3,2021,4,"1,657,615","5,984,240","2,072,236","523,107","252,596","10,489,794",Oct-Dic 2021
4,2021,3,"1,950,631","5,826,257","1,566,048","492,415","250,190","10,085,541",Jul-Sept 2021


In [67]:
nacional_tipo_conexion_economia = total_nacional_accesos_internet_fijo_por_tecnologia_df.drop(columns="Periodo") \
.merge(ingresos_por_operación_servicio_internet_fijo_df.drop(columns="Periodo"), how="left", on=["Año", "Trimestre"]) \
.merge(indicadores_macroeconomicos_df, how="left", on=["Año", "Trimestre"]) \

nacional_tipo_conexion_economia.head()

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Ingresos (miles de pesos),IPC US,IPC AR,USDARS oficial,USDARS blue,PBI millones
0,2022,3,"1,395,277","6,031,970","2,871,541","557,110","235,230","11,091,128",67055930,NaN,NaN,NaN,NaN,NaN
1,2022,2,"1,468,333","5,979,214","2,723,285","556,243","231,609","10,958,684",60335724,NaN,NaN,NaN,NaN,NaN
2,2022,1,"1,533,240","6,073,426","2,219,533","545,814","251,996","10,624,009",55589997,NaN,NaN,NaN,NaN,NaN
3,2021,4,"1,657,615","5,984,240","2,072,236","523,107","252,596","10,489,794",45467887,NaN,NaN,NaN,NaN,NaN
4,2021,3,"1,950,631","5,826,257","1,566,048","492,415","250,190","10,085,541",42999944,NaN,NaN,NaN,NaN,NaN


In [68]:
nacional_tipo_conexion_economia = nacional_tipo_conexion_economia.replace(",", "", regex=True) # need regex=True to find commas into the numbers

In [69]:
nacional_tipo_conexion_economia.to_csv('../processed_data/nacional_tipo_conexion_economia.csv', index=False)

There is no economic information at regional level. The dataframe will be saved as it is

In [76]:
acceso_internet_fijo_por_tecnologia_provincia_df.head()

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2022,3,Buenos Aires,"339,648","2,748,325","1,436,433","126,846","70,416","4,721,668"
1,2022,3,Capital Federal,"140,791","1,240,125","129,218","5,758","31,787","1,547,679"
2,2022,3,Catamarca,"10,010","10,495","46,224","1,329","2,235","70,293"
3,2022,3,Chaco,"27,164","61,800","44,645","8,178","2,359","144,146"
4,2022,3,Chubut,"45,377","72,212","9,574","29,784","8,831","165,778"


In [77]:
acceso_internet_fijo_por_tecnologia_provincia_df.to_csv('../processed_data/regional_tipo_conexion.csv', index=False)

## Datos a nivel localidad

Este conjunto de datos contiene información sobre toda la nación, localidades, partidos y provincias. El alcance de este proyecto incluye solo datos nacionales y provinciales, lo que significa que se ignorarán las divisiones territoriales menores, ya que no serán útiles en el análisis general del estado de las telecomunicaciones del país. Tal vez, en el futuro, un estudio más enfocado podría usar estas divisiones administrativas y obtener más información a nivel local.

In [20]:
accesos_internet_fijo_por_tecnología_localidad_df.head()

,Provincia,Partido,Localidad,Link Indec,ADSL,CABLEMODEM,DIAL UP,FIBRA OPTICA,OTROS,SATELITAL,WIMAX,WIRELESS,Total general,
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,873,"4,704",- 0,2,- 0,975,- 0,664,"7,218",
1,BUENOS AIRES,25 de Mayo,Del Valle,6854020,181,- 0,- 0,- 0,10,1,- 0,- 0,192,
2,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,6854040,- 0,- 0,- 0,- 0,- 0,- 0,- 0,181,181,
3,BUENOS AIRES,25 de Mayo,Norberto de la Riestra,6854060,- 0,782,- 0,6,167,- 0,- 0,327,"1,282",
4,BUENOS AIRES,25 de Mayo,Lucas Monteverde,6854050,- 0,- 0,- 0,- 0,- 0,- 0,- 0,6,6,


In [21]:
accesos_internet_fijo_por_velocidad_localidad_df.head()

,Provincia,Partido,Localidad,Link Indec,Otros,"0,256 mbps","0,375 mbps","0,5 mbps","0,512 mbps","0,625 mbps",...,75 mbps,78 mbps,80 mbps,82 mbps,83 mbps,85 mbps,90 mbps,92 mbps,95 mbps,100 mbps
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,,,,2,,,...,,,9,,,,,,,
1,BUENOS AIRES,25 de Mayo,Agustín Mosconi,6854010,,,,,,,...,,,,,,,,,,
2,BUENOS AIRES,25 de Mayo,Del Valle,6854020,,,,1,,,...,,,,,,,,,,
3,BUENOS AIRES,25 de Mayo,Ernestina,6854030,,,,,,,...,,,,,,,,,,
4,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,6854040,,,,,,,...,,,,,,,,,,


## Datos de conectividad

Los dataframes que implican conectividad por ubicación tienen algunos problemas, como columnas sin nombre y muchos espacios en blanco y valores nulos, por lo que no se utilizarán porque no agregarán valor al análisis.

In [10]:
listado_localidades_conectividad_internet_df.head()

,,Partido,Localidad,ADSL,,DIALUP,FIBRAOPTICA,4G,,TELEFONIAFIJA,,SATELITAL
0,,Bahía Blanca,Villa Bordeau,,,--,--,,,--,,--
1,BUENOS AIRES,,Villa Espora,--,--,,--,--,--,,--,
2,BUENOS AIRES,,,SI,SI,,,SI,SI,,SI,
3,BUENOS AIRES,Balcarce,,--,--,--,,--,--,SI,SI,--
4,BUENOS AIRES,,Napaleofú,--,--,,SI,SI,SI,,,--


In [12]:
listado_localidades_conectividad_internet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4312 entries, 0 to 4311
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0                  4312 non-null   object
 1   Partido        4312 non-null   object
 2   Localidad      4312 non-null   object
 3   ADSL           4312 non-null   object
 4                  4312 non-null   object
 5   DIALUP         4312 non-null   object
 6   FIBRAOPTICA    4312 non-null   object
 7   4G             4312 non-null   object
 8                  4312 non-null   object
 9   TELEFONIAFIJA  4312 non-null   object
 10                 4312 non-null   object
 11  SATELITAL      4312 non-null   object
dtypes: object(12)
memory usage: 404.4+ KB


In [13]:
conectividad_al_servicio_de_internet_df.head()

,,Partido,Localidad,,,CABLEMODEM,DIALUP,,,WIRELESS,TELEFONIAFIJA,,,link,Latitud,
0,,Bahía Blanca,Villa Bordeau,0,,--,--,--,,--,--,,SI,6056020,"-38,6472605094596",
1,BUENOS AIRES,,,0,--,,,--,--,,,--,--,,"-38,7765069529222","-62,1851833537179"
2,BUENOS AIRES,,Balcarce,38376,SI,,SI,SI,--,,,SI,SI,6063010,,"-58,2551665841248"
3,BUENOS AIRES,,,337,--,,,--,--,,SI,,--,6063020,"-37,9412057603",
4,BUENOS AIRES,Balcarce,,374,--,--,,SI,--,SI,SI,SI,,6063030,"-37,6254980210026","-58,7461862359423"


In [14]:
conectividad_al_servicio_de_internet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4312 entries, 0 to 4311
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0                  4312 non-null   object
 1   Partido        4312 non-null   object
 2   Localidad      4312 non-null   object
 3                  4312 non-null   object
 4                  4312 non-null   object
 5   CABLEMODEM     4312 non-null   object
 6   DIALUP         4312 non-null   object
 7                  4312 non-null   object
 8                  4312 non-null   object
 9   WIRELESS       4312 non-null   object
 10  TELEFONIAFIJA  4312 non-null   object
 11                 4312 non-null   object
 12                 4312 non-null   object
 13  link           4312 non-null   object
 14  Latitud        4312 non-null   object
 15                 4312 non-null   object
dtypes: object(16)
memory usage: 539.1+ KB
